In [2]:
import numpy as np
import copy
from IPython.display import display, Markdown

import operator as op
from functools import reduce

# Matrix Method
## Discussion of Theory

To illustrate the Simplex algorithm, consider the following LP.

> Maximize $z = 4x_1 + 2x_2 + x_3$
>
> s.t.
>
> $x_1 \leq 1$
> 
> $4x_1 + x_2 + 6x_3 \leq 6$
>
> $8x_1 + 4x_2 + x_3 \leq 36$
>
> $x_1, x_2, x_3 \geq 0$

### Step 0: Reformat into Standard Form.
Remember that "Standard Form" for an LP is the following.

Max $z = \vec{c}^T \vec{x}$

s.t.

$\vec{\vec{A}} \vec{x} = \vec{b}$

$\vec{x} \geq 0$

We can reformat the original problem into this form using slack variables $s_1$, $s_2$, & $s_3$

> Maximize $z = 4x_1 + 2x_2 + x_3$
>
> s.t.
>
> $x_1 + s_1 = 1$
> 
> $4x_1 + x_2 + 6x_3 + s_3 = 6$
>
> $8x_1 + 4x_2 + x_3 + s_4 = 36$
>
> $x_1, x_2, x_3, s_1, s_2, s_3 \geq 0$

The standard matrix and vectors for this reformatted problem are as follows.

> $\vec{c}^T = \begin{pmatrix}
4 & 2 & 1 & 0 & 0 & 0\\
\end{pmatrix}$
>
> $\vec{\vec{A}} = \begin{pmatrix}
1 & 0 & 0 & 1 & 0 & 0\\
4 & 1 & 6 & 0 & 1 & 0\\
8 & 4 & 1 & 0 & 0 & 1
\end{pmatrix}$
>
> $\vec{x}^T = \begin{pmatrix}
x_1 & x_2 & x_3 & s_1 & s_2 & s_3\\
\end{pmatrix}$
>
> $\vec{b}^T = \begin{pmatrix}
1 & 6 & 36\\
\end{pmatrix}$

### Step 1: Guess a Basis
Now we can begin the actual Simplex algorithm.

Remember that the gist of the simplex algorithm is that an optimal solution to an LP will lie at a vertex of two or more constraints. The trick is to find which vertex is the optimal solution. For any randomly selected vertex we could check if we have the optimal solution by checking each neighboring vertex and comparing objective function values. If we found a neighboring vertex with a more optimal objective function value, then we try again on that vertex. If, by chance we picked the optimal vertex right off the bat, we could prove that we have the optimal solution relatively quickly. But if we already knew the optimal solution, we wouldn't need the Simplex algorithm in the first place. So, instead we select a random vertex and iterate from there until we find the optimal vertex.

Mathematically, we define a vertex as a series of active inequality constraints. In standard form, an active inequality constraint simply means that a respective variable is set to zero. In other words, selecting the appropriate active constraints is mathematically equivalent to setting appropriate variables to zero and solving the linear equation to find the values of the remaining variables. We call the remaining variables a "Basis" and the zero-values variables "Basic Variables".

In matrix form, it looks like this.
$\vec{\vec{A}} = \begin{pmatrix}
\vec{\vec{B}}\  \vert \vec{\vec{N}}\\
\end{pmatrix}$

$\vec{\vec{A}} \vec{x} = \vec{\vec{B}} x_B + \vec{\vec{N}} x_N = \vec{b}$

Where $\vec{\vec{B}}$ is an $m \times m$ matrix of the parts of $\vec{\vec{A}}$ pertaining the the non-basic varibles ($x_B$). Likewise $\vec{\vec{N}}$ is an $m \times (n-m)$ matrix of the parts of $\vec{\vec{A}}$ pertaining the the basic varibles ($x_N$). Where $m$ is the number of equality constraints and $n$ is the number of variables in the standard form problem.

Similarly, $\vec{c}$ can be split into appropriate sub vectors $\vec{c}_B$ and $\vec{c}_N$ containing the coefficients of $\vec{c}$ for the non-basic and basic variables respectively.

In our example, I'll guess that $x_1$, $x_2$, & $x_3$ are the basic variables. The resulting matrices and vectors are as follows.

> $\vec{\vec{B}} = \begin{pmatrix}
1 & 0 & 0\\
0 & 1 & 0\\
0 & 0 & 1
\end{pmatrix}$
>
> $\vec{\vec{N}} = \begin{pmatrix}
1 & 0 & 0\\
4 & 1 & 6\\
8 & 4 & 1
\end{pmatrix}$
>
> $\vec{x}_B^T = \begin{pmatrix}
s_1 & s_2 & s_3
\end{pmatrix}$
>
> $\vec{x}_N^T = \begin{pmatrix}
x_1 & x_2 & x_3
\end{pmatrix}$
>
> $\vec{c}_B^T = \begin{pmatrix}
0 & 0 & 0
\end{pmatrix}$
>
> $\vec{c}_N^T = \begin{pmatrix}
4 & 2 & 1
\end{pmatrix}$

### Step 2: Evaluate Optimality

Remember that, in order for the solution to be optimal, each neighboring vertex must have a worse objective function vaule than the chosen vertex. In other words, the objective function "cost" for switching from one vertex to each of it's neighbors must be positive. Fortunatley, the cost of switching vertices is easy to calculte. The formula for this "cost" (typically called reduced cost) is as follows.
$$RC = -\vec{c}_B^T \ \vec{\vec{B}}^{-1} \ \vec{\vec{N}}  + \vec{c}_N^T \leq \vec{0}^T$$

If this condition is met, then you've proven that the current vertex, with it's accompanying solution $\left( \vec{x}_B = \vec{\vec{B}}^{-1}\vec{b} \ , \ \vec{x}_N = \vec{0} \right)$ is optimal.

In the case of our example, the reduced cost vector is as follows.
> $RC = \begin{pmatrix}
4 & 2 & 1\\
\end{pmatrix} \nleq \vec{0}^T$

So we know our current solution is not optimal.

### Step 3: Determine how to swap variables into and out of the basis

Technically, you could pick any algorithm to swap variables into and out of the basis: So long as the condition in step 2 is satisfied, you can arrive at the optimal solution. But randomly moving variables around is not likely to get you to the optimal solution in a reasonable amount of time. Here, I'll present some common heuristics (rules of thumb) for determining which variables to move in and out. Profesional solvers are exceptionally good at knowing which variables to move in and out; that is why they're so fast.

#### Step 3-a: Select which variable to move from B to N
Since we want our reduced costs to be less than or equal to zero, a good choice to eliminate is the variable whose associated reduced cost is the highest.

In our example, the highest reduced cost value is $4$ which corresponds to $s_1$ in $\vec{x}_B$. So we'll select $s_1$ to move from B to N.

#### Step 3-b: Select which variable to move from N to B
Determining which incoming variable will produce the lowest reduced cost is a little more tricky and mathematically involved. But the gist is that you want to select the variable $i$ with the lowest positive value of $\frac{\left( \vec{\vec{B}}^{-1} \vec{b} \right)_i}{\left( \vec{\vec{B}}^{-1} \vec{\vec{N}} \right)_{i,j}}$ where $j$ is the variable selected in step 3-a.

In our example, $x_1$ produced this lowest positive value. So ultimately we will swap the matrix and vector coefficients of $s_1$ and $x_1$.

### Step 4: Repeat
One the appropriate swaps determined in step 3 and completed and the new matrices and vectors are assembled accordingly, return to step 2 and repeat.

## Python Implementation

In [3]:
# From the description above (see the end of Step 1)

B = np.array([
    [1,0,0],
    [0,1,0],
    [0,0,1]
])
N = np.array([
    [1,0,0],
    [4,1,6],
    [8,4,1]
])

xB = np.array(["s1","s2","s3"])
xN = np.array(["x1","x2","x3"])

cB = np.array([
    [0,],
    [0,],
    [0,]
])
cN = np.array([
    [4,],
    [2,],
    [1,]
])

b = np.array([
    [1,],
    [6,],
    [36,]
])

def computeRC(cB,B,N,cN):
    return -np.matmul(np.matmul(np.transpose(cB),np.linalg.inv(B)),N) + np.transpose(cN)

def isOptimal(RC):
    for n in np.squeeze(RC): #squeeze simply reduces the matrix to vector
        if n > 0:
            return False
    return True

RC = computeRC(cB,B,N,cN)

while not isOptimal(RC):
    #Determine variables to swap
    BtoNIndex = np.argmax(RC)
    
    Binv = np.linalg.inv(B) #Note that this is a computationally expensive operation.
    Binvb = np.matmul(Binv,b)
    BinvN = np.matmul(Binv,N)
    
    testArray = np.array([Binvb[i] / BinvN[i,BtoNIndex] for i in range(len(B))])
    NtoBIndex = np.where(testArray > 0, testArray, np.inf).argmin()
    
    #SwapVariables
    temp = copy.deepcopy(cB[BtoNIndex,0])
    cB[BtoNIndex,0] = cN[NtoBIndex,0]
    cN[NtoBIndex,0] = temp
    
    temp = copy.deepcopy(xB[BtoNIndex])
    xB[BtoNIndex] = xN[NtoBIndex]
    xN[NtoBIndex] = temp
    
    temp = copy.deepcopy(B[:,BtoNIndex])
    B[:,BtoNIndex] = N[:,NtoBIndex]
    N[:,NtoBIndex] = temp
    
    RC = computeRC(cB,B,N,cN)
    
#Compile the results
xBValues = np.matmul(np.linalg.inv(B),b)

allVars = {
    "x1": None,
    "x2": None,
    "x3": None,
    "s1": None,
    "s2": None,
    "s3": None
}

for i in range(len(xB)):
    varName = xB[i]
    value = xBValues[i,0]
    allVars[varName] = value 
    
for i in range(len(xN)):
    varName = xN[i]
    value = 0.0
    allVars[varName] = value 
    
displayVars = ["x1","x2","x3"]
    
for var in displayVars:
    print("{}: {}".format(var,allVars[var]))

x1: 0.0
x2: 6.0
x3: 0.0


C:\Users\nb5786-A\AppData\Local\Temp\ipykernel_10712\1872443589.py:53: RuntimeWarning: divide by zero encountered in divide
  testArray = np.array([Binvb[i] / BinvN[i,BtoNIndex] for i in range(len(B))])


# Tableau Method

### Step 0: Reformat into Standard Form

See step 0 of the Matrix Method, it's the same thing.

I'll start with a slightly different example problem already written in standard form

> Maximize $z = 4x_1 + 2x_2 + x_3$
>
> s.t.
>
> $2x_1 + s_1 = 1$
> 
> $4x_1 + x_2 + 6x_3 + s_2 = 6$
>
> $8x_1 + 4x_2 + x_3 + s_3 = 36$
>
> $x_1, x_2, x_3, s_1, s_2, s_3 \geq 0$

### Step 1: Assuemble Tableau

Write each descision variable and the objective variable (z) in a row on the top of the tableau.

For each constraint, write the coefficient of each variable in a row leaving the constant term off to the right.

On the last line, write the coefficients of the objective function (reformatted as $0 = z - \vec{c}^T \vec{x}$).

Draw a horizontal line sectioning off the objective function row and a vertical line sectioning off the constant column.

Finally, place the slack variables in a column to the left of the tableau, each one in line with it's respective constraint.

For our example the tableau should look like this:

$\begin{array}{c|ccccccc|c}
BASIS      & x_1 & x_2  &  x_3 & s_1 & s_2 & s_3  & z  & CONSTANT \\
\hline s_1 & 2   & 0    & 0    & 1   & 0   & 0    & 0  & 1        \\
       s_2 & 4   & 1    & 6    & 0   & 1   & 0    & 0  & 6        \\
       s_3 & 8   & 4    & 1    & 0   & 0   & 1    & 0  & 36       \\
\hline     & -4  & -2   & -1   & 0   & 0   & 0    & 1  & 0        \\
\end{array}$

### Step 2: Check Optimality
If there are no negative values remaining in the bottom row of the tableau (meaning the reduced cost of any pivot is positive, worsening the objective function value), then the tableau is currently showing an optimal solution.

### Step 3: Pivot Tableau
#### Step 3-a: Assign a Pivot Column
The column with the most negative value in the bottom row is the pivot column. This represents the variable with the most impactful reduced cost for swapping into or out of the basis.

$\begin{array}{c|ccccccc|c}
BASIS      & x_1 & x_2  &  x_3 & s_1 & s_2 & s_3  & z  & CONSTANT \\
\hline s_1 & 2   & 0    & 0    & 1   & 0   & 0    & 0  & 1        \\
       s_2 & 4   & 1    & 6    & 0   & 1   & 0    & 0  & 6        \\
       s_3 & 8   & 4    & 1    & 0   & 0   & 1    & 0  & 36       \\
\hline     & -4  & -2   & -1   & 0   & 0   & 0    & 1  & 0        \\
           & \uparrow & &      &     &     &      &    &          \\
           & Pivot\ Column & &  &     &     &      &    &         \\
\end{array}$

#### Step 3-b: Assign a Pivot Row
For each constraint (row in the tableau except the bottom one) divide the contstant value (right-most column) by the value in the pivot column (from previous step) and respetive row. Write this value to the right of the tableau. Then, the row with the smallest value is the pivot row.

$\begin{array}{c|ccccccc|c}
BASIS      & x_1 & x_2  &  x_3 & s_1 & s_2 & s_3  & z  & CONSTANT \\
\hline s_1 & 2   & 0    & 0    & 1   & 0   & 0    & 0  & 1        \\
       s_2 & 4   & 1    & 6    & 0   & 1   & 0    & 0  & 6        \\
       s_3 & 8   & 4    & 1    & 0   & 0   & 1    & 0  & 36       \\
\hline     & -4  & -2   & -1   & 0   & 0   & 0    & 1  & 0        \\
           & \uparrow & &      &     &     &      &    &          \\
\end{array}$
$\begin{array}{c}
 \\
0.5 \\
1.5 \\
4.5 \\
\\
\\
\end{array}$
$\begin{array}{c}
\\
\leftarrow Pivot\ Row\\
\\
\\
\\
\\
\end{array}$

#### Step 3-c: Identify the Pivot Value
The value in the pivot column and pivot row is the pivot row.

$\begin{array}{c|ccccccc|c}
BASIS      & x_1 & x_2  &  x_3 & s_1 & s_2 & s_3  & z  & CONSTANT \\
\hline s_1 & *2*   & 0    & 0    & 1   & 0   & 0    & 0  & 1        \\
       s_2 & 4   & 1    & 6    & 0   & 1   & 0    & 0  & 6        \\
       s_3 & 8   & 4    & 1    & 0   & 0   & 1    & 0  & 36       \\
\hline     & -4  & -2   & -1   & 0   & 0   & 0    & 1  & 0        \\
           & \uparrow & &      &     &     &      &    &          \\
\end{array}$
$\begin{array}{c}
\\
\leftarrow\\
\\
\\
\\
\\
\end{array}$

#### Step 3-d: Row Reduction Operation #1
Divide the entire pivot row by the pivot value.

$\begin{array}{c|ccccccc|c}
BASIS      & x_1 & x_2  &  x_3 & s_1 & s_2 & s_3  & z  & CONSTANT \\
\hline s_1 & *1* & 0    & 0    & 1/2 & 0   & 0    & 0  & 1/2      \\
       s_2 & 4   & 1    & 6    & 0   & 1   & 0    & 0  & 6        \\
       s_3 & 8   & 4    & 1    & 0   & 0   & 1    & 0  & 36       \\
\hline     & -4  & -2   & -1   & 0   & 0   & 0    & 1  & 0        \\
           & \uparrow & &      &     &     &      &    &          \\
\end{array}$
$\begin{array}{c}
\\
\leftarrow\\
\\
\\
\\
\\
\end{array}$

#### Step 3-d: Row Reduction Operation #2
Use row reduction to ensure that all other values (other than the pivot value which is now 1) are zero. If you don't know what that means, that's okay, I'll write the pseudo-code below.

```
pi = the index of the pivot row
pj = the index of the pivot column

for i in rowIndicies(except pi):
    multiplier_i = tableau[i,pj]
    for j in columnIndicies:
        tableau[i,j] -= multiplier_i * tableau[pi,j]
```

Then, replace the Basis variable of the pivot row with the variable assicated with the pivot column.

Doing so, in our example, produces the following updated tableau

$\begin{array}{c|ccccccc|c}
BASIS      & x_1 & x_2  &  x_3 & s_1 & s_2 & s_3  & z  & CONSTANT \\
\hline x_1 & *1* & 0    & 0    & 1/2 & 0   & 0    & 0  & 1/2      \\
       s_2 & 0   & 1    & 6    & -2  & 1   & 0    & 0  & 4        \\
       s_3 & 0   & 4    & 1    & -4  & 0   & 1    & 0  & 32       \\
\hline     & 0   & -2   & -1   & 2   & 0   & 0    & 1  & 2        \\
           & \uparrow & &      &     &     &      &    &          \\
\end{array}$
$\begin{array}{c}
\\
\leftarrow\\
\\
\\
\\
\\
\end{array}$

#### Step 3-e: Repeat
Go back to step 2 and repeat unless and optimal solution has been reached.

### Step 4: Interpetation
The variables listed to the left of the tableau form the "basis". This means that their optimal values are shown in their respective row of the constant column. Any variable not in the basis has a value of zero at the optimal solution.

The value shown in the bottom-right cell of the tableau is the optimal objective function value.

## Python Implementation

In [18]:
variables = ["x1","x2","x3","s1","s2","s3"]
basisVariables = ["s1","s2","s3"]

numConstraints = 3

tableauBody = [
    [2.,0.,0.,1.,0.,0.,1. ],
    [4.,1.,6.,0.,1.,0.,6. ],
    [8.,4.,1.,0.,0.,1.,36.],
    [-4.,-2.,-1.,0.,0.,0.,0.]
]

def optimalSolutionFound():
    for x in tableauBody[-1]:
        if x < -1e-7: #The 1e-7 is to prevent computer rounding error
            return False
    return True

def printTableau(pivotIndices=None):
    markdownStr = "$\\begin{array}{c|cccccc|c}\n"
    markdownStr += "BASIS      & x_1 & x_2  &  x_3 & s_1 & s_2 & s_3 & CONSTANT \\\\\n\\hline "
    for i in range(numConstraints):
        markdownStr += basisVariables[i]
        for j in range(len(tableauBody[0])):
            if pivotIndices != None and i == pivotIndices[0] and j == pivotIndices[1]:
                markdownStr += " & *" + str(tableauBody[i][j]) + "*"
            else:
                markdownStr += " & " + str(tableauBody[i][j])
        markdownStr += "\\\\\n"
    markdownStr += "\\hline"
    for j in range(len(tableauBody[0])):
        markdownStr += " & " + str(tableauBody[-1][j])
    markdownStr += "\\\\\n\\end{array}$"
    
    display(Markdown(markdownStr))

itrNumber = 0
maxItr = 5

while not optimalSolutionFound() and itrNumber < maxItr:
    itrNumber += 1
    pj = np.argmin(tableauBody[-1][:-1])

    testValues = [tableauBody[i][-1] / tableauBody[i][pj] if tableauBody[i][pj] != 0 else np.infty for i in range(numConstraints)]
    testValues = [val if val >= 0 else np.infty for val in testValues]
    pi = np.argmin(testValues)
    
    printTableau([pi,pj])
    
    pivotValue = tableauBody[pi][pj]
    for j in range(len(tableauBody[0])):
        tableauBody[pi][j] /= pivotValue
        
    for i in range(len(tableauBody)):
        if i == pi:
            continue
        
        multiplier = tableauBody[i][pj]
        for j in range(len(tableauBody[0])):
            tableauBody[i][j] -= multiplier * tableauBody[pi][j]
            
    basisVariables[pi] = variables[pj]
            
if maxItr == itrNumber:
    print("Optimization terminated after reaching maximum number of iterations...\nFinal Tableau:")
else:
    print("Optimal Solution:")
printTableau()

            
#WHY ISN'T THE PIVOT STARS SHOWING UP  EVERY TIME? COULD BE THE CAUSE OF THE BUG HERE.

$\begin{array}{c|cccccc|c}
BASIS      & x_1 & x_2  &  x_3 & s_1 & s_2 & s_3 & CONSTANT \\
\hline s1 & *2.0* & 0.0 & 0.0 & 1.0 & 0.0 & 0.0 & 1.0\\
s2 & 4.0 & 1.0 & 6.0 & 0.0 & 1.0 & 0.0 & 6.0\\
s3 & 8.0 & 4.0 & 1.0 & 0.0 & 0.0 & 1.0 & 36.0\\
\hline & -4.0 & -2.0 & -1.0 & 0.0 & 0.0 & 0.0 & 0.0\\
\end{array}$

$\begin{array}{c|cccccc|c}
BASIS      & x_1 & x_2  &  x_3 & s_1 & s_2 & s_3 & CONSTANT \\
\hline x1 & 1.0 & 0.0 & 0.0 & 0.5 & 0.0 & 0.0 & 0.5\\
s2 & 0.0 & *1.0* & 6.0 & -2.0 & 1.0 & 0.0 & 4.0\\
s3 & 0.0 & 4.0 & 1.0 & -4.0 & 0.0 & 1.0 & 32.0\\
\hline & 0.0 & -2.0 & -1.0 & 2.0 & 0.0 & 0.0 & 2.0\\
\end{array}$

$\begin{array}{c|cccccc|c}
BASIS      & x_1 & x_2  &  x_3 & s_1 & s_2 & s_3 & CONSTANT \\
\hline x1 & 1.0 & 0.0 & 0.0 & *0.5* & 0.0 & 0.0 & 0.5\\
x2 & 0.0 & 1.0 & 6.0 & -2.0 & 1.0 & 0.0 & 4.0\\
s3 & 0.0 & 0.0 & -23.0 & 4.0 & -4.0 & 1.0 & 16.0\\
\hline & 0.0 & 0.0 & 11.0 & -2.0 & 2.0 & 0.0 & 10.0\\
\end{array}$

Optimal Solution:


$\begin{array}{c|cccccc|c}
BASIS      & x_1 & x_2  &  x_3 & s_1 & s_2 & s_3 & CONSTANT \\
\hline s1 & 2.0 & 0.0 & 0.0 & 1.0 & 0.0 & 0.0 & 1.0\\
x2 & 4.0 & 1.0 & 6.0 & 0.0 & 1.0 & 0.0 & 6.0\\
s3 & -8.0 & 0.0 & -23.0 & 0.0 & -4.0 & 1.0 & 12.0\\
\hline & 4.0 & 0.0 & 11.0 & 0.0 & 2.0 & 0.0 & 12.0\\
\end{array}$

# Example of Multiple Optimal Solutions

Consider the following LP:

Max $z = \vec{c}^T \vec{x}$

s.t.

$\vec{\vec{A}} \vec{x} = \vec{b}$

$\vec{x} \geq 0$

Where

$\vec{x}^T = \begin{pmatrix}
x_1 & x_2 & x_3 & s_1 & s_2 & s_3 & u_{1,p} & u_{1,m} & u_{2,p} & u_{2,m} & u_{3,p} & u_{3,m}
\end{pmatrix}$

$\vec{c}^T = \begin{pmatrix}
0 & 0 & 0 & 0 & 0 & 0 & 1 & 1 & 1 & 1 & 1 & 1
\end{pmatrix}$

$\vec{\vec{A}} = \begin{pmatrix}
2 & -1 & 0 & 1 & 0 & 0 & 1 & -1 & 0 & 0 & 0 & 0\\
-2 & 1 & 0 & 0 & 1 & 0 & 0 & 0 & 1 & -1 & 0 & 0\\
-1 & 1 & 1 & 0 & 0 & 1 & 0 & 0 & 0 & 0 & 1 & -1
\end{pmatrix}$

$\vec{b}^T = \begin{pmatrix}
4 & -5 & -1
\end{pmatrix}$

Using the simplex method as coded above, but printing the $RC$ vector at each iteration produces the following:

In [40]:
variables = ["x1","x2","x3","s1","s2","s3","u1p","u1m","u2p","u2m","u3p","u3m"]
basisVariables = ["u1p","u1m","u2m"]

numConstraints = 3

tableauBody = [
    [ 2,-1, 0, 1, 0, 0, 1,-1, 0, 0, 0, 0, 4],
    [-2, 1, 0, 0, 1, 0, 0, 0, 1,-1, 0, 0,-5],
    [-1, 1, 1, 0, 0, 1, 0, 0, 0, 0, 1,-1,-1],
    [ 0, 0, 0, 0, 0, 0,-1,-1,-1,-1,-1,-1, 0]
]

class StringSearchTree:
    def __init__(self):
        self.childNodes = {}
    def searchCombo(self,combo):
        if len(combo) == 1:
            if combo[0] in self.childNodes:
                return True
            else:
                self.childNodes[combo[0]] = StringSearchTree()
                return False
        else:
            if combo[0] not in self.childNodes:
                self.childNodes[combo[0]] = StringSearchTree()
            return self.childNodes[combo[0]].searchCombo(combo[1:])
                
    

def optimalSolutionFound():
    for x in tableauBody[-1]:
        if x < -1e-7: #The 1e-7 is to prevent computer rounding error
            return False
    return True

def printTableau(pivotIndices=None):
    markdownStr = "$\\begin{array}{c|cccccccccccc|c}\n"
    markdownStr += "BASIS      & x_1 & x_2  &  x_3 & s_1 & s_2 & s_3 & u_{1+} & u_{1-} & u_{2+} & u_{2-} & u_{3+} & u_{3-} & CONSTANT \\\\\n\\hline "
    for i in range(numConstraints):
        markdownStr += basisVariables[i]
        for j in range(len(tableauBody[0])):
            if pivotIndices != None and i == pivotIndices[0] and j == pivotIndices[1]:
                markdownStr += " & *" + str(tableauBody[i][j]) + "*"
            else:
                markdownStr += " & " + str(tableauBody[i][j])
        markdownStr += "\\\\\n"
    markdownStr += "\\hline"
    for j in range(len(tableauBody[0])):
        markdownStr += " & " + str(tableauBody[-1][j])
    markdownStr += "\\\\\n\\end{array}$"
    
    display(Markdown(markdownStr))

def ncr(n,r):
    r = min(r,n-r)
    numer = reduce(op.mul, range(n,n-r,-1),1)
    denom = reduce(op.mul, range(1,r+1),1)
    return numer // denom
    
itrNumber = 0
maxItr = ncr(len(variables), numConstraints)

previousBases = StringSearchTree()
#previousBases.searchCombo(basisVariables)

while not optimalSolutionFound() and itrNumber < maxItr:
    itrNumber += 1
    RCs = [[i,tableauBody[-1][i]] for i in range(len(tableauBody[-1])-1)]
    RCs.sort(key=lambda x: x[1])
    for RCtestIndex in range(len(RCs)):
        #Generate a propsed basis
        pj = RCs[RCtestIndex][0]
        RC = RCs[RCtestIndex][1]
        if RC > 0:
            raise Exception("Error! There are no more unique bases with a negative RC!")

        print(RCtestIndex,pj,RC)
        testValues = [tableauBody[i][-1] / tableauBody[i][pj] if tableauBody[i][pj] != 0 else np.infty for i in range(numConstraints)]
        testValues = [val if val >= 0 else np.nan for val in testValues]
        numInf = np.sum(np.isnan(testValues) + np.isposinf(testValues) + np.isneginf(testValues))
        print(testValues,numInf)
        if numInf == numConstraints:
            continue
        pi = np.argmin(testValues)
        
        testBasis = copy.deepcopy(basisVariables)
        testBasis[pi] = variables[pj]
        
        #See if this basis has already been visited
        #If not, this is an acceptable basis
        if not previousBases.searchCombo(testBasis):
            break
        else:
            if RCtestIndex == len(RCs) - 1:
                raise Exception("Error! There are no more unique bases!")
    
    printTableau([pi,pj])
    
    pivotValue = tableauBody[pi][pj]
    for j in range(len(tableauBody[0])):
        tableauBody[pi][j] /= pivotValue
        
    for i in range(len(tableauBody)):
        if i == pi:
            continue
        
        multiplier = tableauBody[i][pj]
        for j in range(len(tableauBody[0])):
            tableauBody[i][j] -= multiplier * tableauBody[pi][j]
            
    basisVariables[pi] = variables[pj]
            
if maxItr == itrNumber:
    print("Optimization terminated after reaching maximum number of iterations (", maxItr,")...\nFinal Tableau:")
else:
    print("Optimal Solution:")
printTableau()


0 6 -1
[4.0, inf, inf] 2


$\begin{array}{c|cccccccccccc|c}
BASIS      & x_1 & x_2  &  x_3 & s_1 & s_2 & s_3 & u_{1+} & u_{1-} & u_{2+} & u_{2-} & u_{3+} & u_{3-} & CONSTANT \\
\hline u1p & 2 & -1 & 0 & 1 & 0 & 0 & *1* & -1 & 0 & 0 & 0 & 0 & 4\\
u1m & -2 & 1 & 0 & 0 & 1 & 0 & 0 & 0 & 1 & -1 & 0 & 0 & -5\\
u2m & -1 & 1 & 1 & 0 & 0 & 1 & 0 & 0 & 0 & 0 & 1 & -1 & -1\\
\hline & 0 & 0 & 0 & 0 & 0 & 0 & -1 & -1 & -1 & -1 & -1 & -1 & 0\\
\end{array}$

0 7 -2.0
[nan, inf, inf] 3
1 1 -1.0
[nan, nan, nan] 3
2 8 -1.0
[inf, nan, inf] 3
3 9 -1.0
[inf, 5.0, inf] 2


$\begin{array}{c|cccccccccccc|c}
BASIS      & x_1 & x_2  &  x_3 & s_1 & s_2 & s_3 & u_{1+} & u_{1-} & u_{2+} & u_{2-} & u_{3+} & u_{3-} & CONSTANT \\
\hline u1p & 2.0 & -1.0 & 0.0 & 1.0 & 0.0 & 0.0 & 1.0 & -1.0 & 0.0 & 0.0 & 0.0 & 0.0 & 4.0\\
u1m & -2.0 & 1.0 & 0.0 & 0.0 & 1.0 & 0.0 & 0.0 & 0.0 & 1.0 & *-1.0* & 0.0 & 0.0 & -5.0\\
u2m & -1.0 & 1.0 & 1.0 & 0.0 & 0.0 & 1.0 & 0.0 & 0.0 & 0.0 & 0.0 & 1.0 & -1.0 & -1.0\\
\hline & 2.0 & -1.0 & 0.0 & 1.0 & 0.0 & 0.0 & 0.0 & -2.0 & -1.0 & -1.0 & -1.0 & -1.0 & 4.0\\
\end{array}$

0 1 -2.0
[nan, nan, nan] 3
1 7 -2.0
[nan, inf, inf] 3
2 8 -2.0
[inf, nan, inf] 3
3 4 -1.0
[inf, nan, inf] 3
4 10 -1.0
[inf, inf, nan] 3
5 11 -1.0
[inf, inf, 1.0] 2


$\begin{array}{c|cccccccccccc|c}
BASIS      & x_1 & x_2  &  x_3 & s_1 & s_2 & s_3 & u_{1+} & u_{1-} & u_{2+} & u_{2-} & u_{3+} & u_{3-} & CONSTANT \\
\hline u1p & 2.0 & -1.0 & 0.0 & 1.0 & 0.0 & 0.0 & 1.0 & -1.0 & 0.0 & 0.0 & 0.0 & 0.0 & 4.0\\
u2m & 2.0 & -1.0 & -0.0 & -0.0 & -1.0 & -0.0 & -0.0 & -0.0 & -1.0 & 1.0 & -0.0 & -0.0 & 5.0\\
u2m & -1.0 & 1.0 & 1.0 & 0.0 & 0.0 & 1.0 & 0.0 & 0.0 & 0.0 & 0.0 & 1.0 & *-1.0* & -1.0\\
\hline & 4.0 & -2.0 & 0.0 & 1.0 & -1.0 & 0.0 & 0.0 & -2.0 & -2.0 & 0.0 & -1.0 & -1.0 & 9.0\\
\end{array}$

0 1 -3.0
[nan, nan, nan] 3
1 7 -2.0
[nan, inf, inf] 3
2 8 -2.0
[inf, nan, inf] 3
3 10 -2.0
[inf, inf, nan] 3
4 2 -1.0
[inf, inf, nan] 3
5 4 -1.0
[inf, nan, inf] 3
6 5 -1.0
[inf, inf, nan] 3
7 6 0.0
[4.0, inf, inf] 2
8 9 0.0
[inf, 5.0, inf] 2
9 11 0.0
[inf, inf, 1.0] 2


Exception: Error! There are no more unique bases with a negative RC!